# Loading Collections from monty-stac-extension Examples

This notebook demonstrates how to load all the collections from the monty-stac-extension examples directory into the Montandon API. We'll:

1. Scan the monty-stac-extension/examples directory for collection JSON files
2. Check if each collection exists in the Montandon API
3. Create or update collections as needed
4. Provide a summary of the operations performed

Let's begin by importing the necessary libraries.

In [27]:
# Basic libraries
import os
import json
import glob
from pathlib import Path
import requests

# STAC and pystac-monty
import pystac
from pystac_monty.extension import MontyExtension

# Import STAC helper functions
import sys
sys.path.append('.')
from stac_helpers import (
    check_stac_api_availability,
    check_collection_exists,
    create_collection_from_file,
    create_collection_fallback,
    delete_collection
)

## Define a function to update collections using PUT

First, let's define a function to update collections using the PUT method, which is not available in the stac_helpers.py file.

In [28]:
def update_collection_from_file(api_url, collection_path) -> bool:
    """
    Update a collection in the STAC API using a predefined collection definition file and the PUT method
    
    Parameters:
    - api_url: URL of the STAC API
    - collection_path: Path to the collection definition file
    
    Returns:
    - Boolean indicating if the collection was updated successfully
    """
    try:
        # Check if the collection definition file exists
        if not os.path.exists(collection_path):
            print(f"Collection definition file not found: {collection_path}")
            return False
        
        # Load the collection definition from the file
        with open(collection_path, 'r') as f:
            collection_dict = json.load(f)
        
        collection_id = collection_dict.get('id')
        if not collection_id:
            print(f"Collection ID not found in {collection_path}")
            return False
        
        # Update the collection using the transaction API with PUT method
        response = requests.put(
            f"{api_url}/collections/{collection_id}",
            json=collection_dict,
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code in [200, 201, 204]:
            print(f"Collection '{collection_id}' updated successfully")
            return True
        else:
            print(f"Failed to update collection '{collection_id}'. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return False
    except Exception as e:
        print(f"Error updating collection from file: {e}")
        return False

## 1. Define the STAC API Endpoint

First, let's define the STAC API endpoint where we'll be creating or updating collections.

In [29]:
# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

if not api_available:
    print("STAC API is not available. Please check the URL and try again.")
    # Exit the notebook or continue with other operations

STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac


## 2. Scan for Collection JSON Files

Now, let's scan the monty-stac-extension/examples directory for collection JSON files.

In [30]:
# Define the path to the monty-stac-extension examples directory
examples_dir = "../monty-stac-extension/examples"

# Find all collection JSON files (they are named after the directory they're in)
collection_files = []
for root, dirs, files in os.walk(examples_dir):
    for file in files:
        if file.endswith(".json") and ("events" in file or "hazards" in file or "impacts" in file):
            collection_files.append(os.path.join(root, file))

print(f"Found {len(collection_files)} collection JSON files")

Found 29 collection JSON files


## 3. Process Each Collection

Now, let's process each collection file and create or update it in the STAC API.

In [31]:
# Function to extract collection ID from file path
def get_collection_id_from_path(file_path):
    return os.path.basename(file_path).replace(".json", "")

# Process each collection file
successful_collections = []
failed_collections = []

for collection_file in collection_files:
    collection_id = get_collection_id_from_path(collection_file)
    print(f"\nProcessing collection: {collection_id}")
    
    # Check if the collection exists
    collection_exists = check_collection_exists(stac_api_url, collection_id)
    
    if collection_exists:
        print(f"Collection '{collection_id}' exists, updating using PUT method...")
        # Update the collection using PUT method
        collection_updated = update_collection_from_file(stac_api_url, collection_file)
        
        if collection_updated:
            successful_collections.append(collection_id)
            print(f"Successfully updated collection '{collection_id}'")
        else:
            failed_collections.append(collection_id)
            print(f"Failed to update collection '{collection_id}'")
    else:
        # Create the collection
        print(f"Collection '{collection_id}' does not exist, creating...")
        collection_created = create_collection_from_file(stac_api_url, collection_file)
        
        if not collection_created:
            print("Trying fallback method to create collection...")
            
            # Load the collection file to get description and roles
            with open(collection_file, 'r') as f:
                collection_data = json.load(f)
            
            description = collection_data.get('description', f"Collection for {collection_id}")
            roles = collection_data.get('roles', [])
            
            collection_created = create_collection_fallback(
                stac_api_url, 
                collection_id, 
                description,
                roles
            )
        
        if collection_created:
            successful_collections.append(collection_id)
            print(f"Successfully created collection '{collection_id}'")
        else:
            failed_collections.append(collection_id)
            print(f"Failed to create collection '{collection_id}'")

# Print summary
print("\n--- Summary ---")
print(f"Successfully processed {len(successful_collections)} collections")
print(f"Failed to process {len(failed_collections)} collections")

if failed_collections:
    print("\nFailed collections:")
    for collection_id in failed_collections:
        print(f"- {collection_id}")


Processing collection: idmc-idu-events
Collection 'idmc-idu-events' exists in the STAC API
Collection 'idmc-idu-events' exists, updating using PUT method...
Collection 'idmc-idu-events' updated successfully
Successfully updated collection 'idmc-idu-events'

Processing collection: usgs-hazards
Collection 'usgs-hazards' exists in the STAC API
Collection 'usgs-hazards' exists, updating using PUT method...
Collection 'usgs-hazards' updated successfully
Successfully updated collection 'usgs-hazards'

Processing collection: ifrcevent-events
Collection 'ifrcevent-events' exists in the STAC API
Collection 'ifrcevent-events' exists, updating using PUT method...
Collection 'ifrcevent-events' updated successfully
Successfully updated collection 'ifrcevent-events'

Processing collection: gdacs-hazards
Collection 'gdacs-hazards' exists in the STAC API
Collection 'gdacs-hazards' exists, updating using PUT method...
Collection 'gdacs-hazards' updated successfully
Successfully updated collection 'gda